In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# Discrimination detection and mitigation (on housing nights multiclass dataset)

## Train a model regardless of fairness

In [ ]:
from fairdream.data_preparation import *
from fairdream.compute_scores import *
from fairdream.detection import *
from fairdream.correction import *
from fairdream.plots import *

from fairdream.multiclass_fair_preparation import *

In [ ]:
# set your statistics purposes
model_task = 'multiclass'
stat_criteria = 'merror'

### Bring your own model

If you want to bring your own model, you have to set 3 features:

1. uncorrected_model_path
Save your model in uncorrected_model_path, for fairness analysis on relevant features
Ex: uncorrected_model_path = "/work/data/models/uncorrected_model.pkl"

2. X_train_valid, multi_Y_train_valid
pd.DataFrame with your inputs and targets on train&valid set, of shape(nb_individuals, nb_labels)

3. multi_predict_proba_train_valid
np.ndarray with the predicted probas by label (i.e. by class), of shape(nb_individuals, nb_labels)

### Automatically train a model statistically performant, regardless of fairness

#### Pre-processing

In [ ]:
# load train and test sets
train_set = automatic_preprocessing("housing_nights_dataset")

In [ ]:
train_set

#### Multiclass model training 

In [ ]:
# entraînement du modèle avec ce nouveau dataset 
# split data into X and y
X = train_set.loc[:, train_set.columns != 'granted_number_of_nights']
Y = train_set.loc[:,'granted_number_of_nights']

In [ ]:
X_train, X_valid, X_train_valid, X_test, multi_Y_train, multi_Y_valid, multi_Y_train_valid, multi_Y_test = train_valid_test_split(X,Y, model_task)

In [ ]:
# save the uncorrected model, to then sort its features by importances
save_model=True
uncorrected_model_path = "/work/data/models/uncorrected_model.pkl"

multi_predict_proba_train_valid = train_naive_xgb(X_train, X_valid, X_train_valid, X_test, multi_Y_train, multi_Y_valid, multi_Y_train_valid, multi_Y_test, model_task, stat_criteria, save_model=save_model)

### Set multiclass to binary, to check discrimination of "better-off" vs "less fortunate" individuals

--- DOCUMENTATION ---
    
    --- Optional parameters for model_task == "multiclass" ---

    When model_task == "multiclass", the fairness is evaluated like in binary classification 
    (!) valid only when Y labels are independant.
    
-- To transform multiclass into 2 classes, the user ranks the labels by ascending order --
   
1. sorted_labels_list

When model_task == "multiclass", list of labels with the desired ascending ranking of the user.
        Ex: when labels are number of housing nights and the user wants to maximise it,
        sorted_labels_list = [0,1,2,3] 
        
-- Then to separate individuals in binary classes, one has 2 choices --

2. frontier_label

To set manually the 'frontier_label' (ex: one chooses that individuals > label "2" nights are privileged)

2. distribution_frontier


To set a % of individuals distribution, i.e. 'distribution_frontier' (median "median", quartiles "Q1" or "Q3") which will automatically determine the 'frontier_label'

In [ ]:
# add a vector with predicted labels 
multi_Y_pred_train_valid = multi_predict_proba_train_valid.argmax(axis=-1)

sorted_labels_list = [0,1,2,3] 

# the user chooses, either a frontier_label or distribution_frontier (to be better documented in fair_detection)
frontier_label = 1

# or: distribution_frontier = 'Q3'

Y_train_valid, Y_pred_train_valid = multi_to_binary_Y_pred(multi_Y_train_valid, multi_Y_pred_train_valid, sorted_labels_list, frontier_label=frontier_label)

## Discrimination detection 

In [ ]:
augment_train_valid_set_with_results("uncorrected", X_train_valid, Y_train_valid, Y_pred_train_valid, model_task, multi_Y_train_valid, multi_predict_proba_train_valid)

In [ ]:
train_valid_set_with_uncorrected_results = augment_train_valid_set_with_results("uncorrected", X_train_valid, Y_train_valid, Y_pred_train_valid, model_task, multi_Y_train_valid, multi_predict_proba_train_valid)

In [ ]:
## Detection alert (on train&valid data to examine if the model learned discriminant behavior)

In [ ]:
augmented_train_valid_set = train_valid_set_with_uncorrected_results
model_name = "uncorrected"

fairness_purpose="true_negative_rate"
injustice_acceptance=1.3
min_individuals_discrimined=0.01

discrimination_alert(augmented_train_valid_set, model_name, fairness_purpose, model_task, injustice_acceptance, min_individuals_discrimined)

## Discrimination correction with a new fair model

### Generating fairer models with weights distorsion

In [ ]:
# the user determines one's fairness objectives to build new fairer models
# on which group and regarding which criteria (purpose, constraint of the models) one aims to erase discrimination

protected_attribute = 'number_of_underage'

# then the user sets the desired balance between stat and fair performances

tradeoff = "moderate"
weight_method = "weighted_groups"
nb_fair_models = 3

train_valid_set_with_corrected_results, models_df, best_model_dict = fair_train(
    X=X,
    Y=Y,
    train_valid_set_with_uncorrected_results=train_valid_set_with_uncorrected_results,
    protected_attribute=protected_attribute,
    fairness_purpose=fairness_purpose,
    model_task=model_task,
    stat_criteria=stat_criteria,
    tradeoff=tradeoff,
    weight_method=weight_method,
    nb_fair_models=nb_fair_models,
    sorted_labels_list=sorted_labels_list,
    # distribution_frontier=distribution_frontier,
    frontier_label=frontier_label,
)

### Evaluating the best fair model

In [ ]:
fair_model_results(train_valid_set_with_corrected_results, models_df, best_model_dict,protected_attribute,fairness_purpose, model_task)

In [ ]:
top_models = models_df.sort_values(by='tradeoff_score',ascending=False)
top_models